In [ ]:
import random
from PIL import Image, ImageDraw
import numpy as np
from scipy.spatial import Voronoi
from skimage.color import deltaE_cie76, rgb2lab
from copy import deepcopy
from multiprocessing import Pool

class ColoredPoint:
    def __init__(self, img_width, img_height):
        self.coordinates = (random.randint(0, img_width), random.randint(0, img_height))
        self.color = (random.randint(0, 256), random.randint(0, 256), random.randint(0, 256), 255)

    def mutate(self, sigma=1.0):
        mutations = ['shift', 'color']
        weights = [50, 50]

        mutation_type = random.choices(mutations, weights=weights, k=1)[0]

        if mutation_type == 'shift':
            self.coordinates = (self.coordinates[0] + int(random.randint(-10, 10) * sigma),
                                self.coordinates[1] + int(random.randint(-10, 10) * sigma))
        elif mutation_type == 'color':
            red = self.color[0] + int(random.randint(-25, 25) * sigma)
            green = self.color[1] + int(random.randint(-25, 25) * sigma)
            blue = self.color[2] + int(random.randint(-25, 25) * sigma)

            self.color = (red, green, blue, 255)
            self.color = tuple(min(max(c, 0), 255) for c in self.color)

class VoronoiPainting:
    def __init__(self, num_points, img_width, img_height):
        self._img_width = img_width
        self._img_height = img_height
        self.points = [ColoredPoint(img_width, img_height) for _ in range(num_points)]

    def draw(self) -> Image:
        image = Image.new("RGBA", (self._img_width, self._img_height), (255, 255, 255, 255))
        draw = ImageDraw.Draw(image)

        coords = [p.coordinates for p in self.points]
        colors = [p.color for p in self.points]

        vor = Voronoi(coords)
        regions, vertices = self.voronoi_finite_polygons_2d(vor)

        for region, color in zip(regions, colors):
            polygon = [tuple(vertices[i]) for i in region if i >= 0]
            if len(polygon) >= 2:  # Ensure valid polygon
                draw.polygon(polygon, fill=color)

        return image

    def mutate(self, mutation_rate=0.01):
        for point in self.points:
            if random.random() < mutation_rate:
                point.mutate()

    @staticmethod
    def voronoi_finite_polygons_2d(vor, radius=None):
        new_regions = []
        new_vertices = vor.vertices.tolist()

        center = vor.points.mean(axis=0)
        if radius is None:
            radius = vor.points.ptp().max()

        all_ridges = {}
        for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
            if v1 < 0 or v2 < 0:
                continue  # Skip infinite ridges
            all_ridges.setdefault(p1, []).append((p2, v1, v2))
            all_ridges.setdefault(p2, []).append((p1, v1, v2))

        for p1, region in enumerate(vor.point_region):
            vertices = vor.regions[region]

            if all(v >= 0 for v in vertices):
                new_regions.append(vertices)
                continue

            ridge_vertices = [v for v in vertices if v >= 0]
            for p2, v1, v2 in all_ridges.get(p1, []):
                if v2 < 0:
                    v = vor.vertices[v1]
                    direction = vor.points[p2] - vor.points[p1]
                    unit_direction = direction / np.linalg.norm(direction)
                    new_vertex = vor.vertices[v1] + unit_direction * radius
                    ridge_vertices.append(len(new_vertices))
                    new_vertices.append(new_vertex.tolist())

            new_regions.append(ridge_vertices)

        return new_regions, np.asarray(new_vertices)

def calculate_fitness(painting, target_image):
    target_array = np.array(target_image.convert('RGB'))
    rendered_image = np.array(painting.draw().convert('RGB'))
    target_lab = rgb2lab(target_array)
    rendered_lab = rgb2lab(rendered_image)
    diff_array = deltaE_cie76(target_lab, rendered_lab)
    return np.mean(diff_array)

def crossover(parent1, parent2):
    child1 = deepcopy(parent1)
    child2 = deepcopy(parent2)

    crossover_point = random.randint(0, len(parent1.points))
    child1.points[crossover_point:], child2.points[crossover_point:] = (
        parent2.points[crossover_point:], parent1.points[crossover_point:])

    return child1, child2

class Population:
    def __init__(self, size, img_width, img_height, num_points, target_image):
        self.size = size
        self.img_width = img_width
        self.img_height = img_height
        self.num_points = num_points
        self.target_image = target_image
        self.individuals = [VoronoiPainting(num_points, img_width, img_height) for _ in range(size)]
        self.fitness = self.calculate_fitness_parallel(self.individuals)

    def calculate_fitness_parallel(self, individuals):
        with Pool() as pool:
            fitness_values = pool.map(calculate_fitness_worker, [(ind, self.target_image) for ind in individuals])
        return fitness_values

    def evolve(self, gens, xo_prob, mut_prob, elitism):
        for gen in range(gens):
            new_pop = []
            if elitism:
                elite = deepcopy(self.individuals[self.fitness.index(min(self.fitness))])

            while len(new_pop) < self.size:
                parent1 = self.tournament_selection()
                parent2 = self.tournament_selection()

                if random.random() < xo_prob:
                    child1, child2 = crossover(parent1, parent2)
                else:
                    child1, child2 = deepcopy(parent1), deepcopy(parent2)

                child1.mutate(mut_prob)
                child2.mutate(mut_prob)

                new_pop.extend([child1, child2])

            if elitism:
                worst_index = self.fitness.index(max(self.fitness))
                new_pop[worst_index] = elite

            self.individuals = new_pop[:self.size]
            self.fitness = self.calculate_fitness_parallel(self.individuals)

            print(f"Generation {gen + 1}, Best Fitness: {min(self.fitness)}")

    def tournament_selection(self, k=3):
        selected = random.sample(list(zip(self.individuals, self.fitness)), k)
        return min(selected, key=lambda x: x[1])[0]

def calculate_fitness_worker(args):
    painting, target_image = args
    return calculate_fitness(painting, target_image)

# Load target image
target_image_path = 'target_image.jpg'
target_image = Image.open(target_image_path).convert('RGB')
img_width, img_height = target_image.size

# Parameters
population_size = 50
num_points = 100
generations = 100
mutation_rate = 0.01
crossover_prob = 0.7
elitism = True

# Initialize and evolve population
population = Population(size=population_size, img_width=img_width, img_height=img_height, num_points=num_points, target_image=target_image)
population.evolve(gens=generations, xo_prob=crossover_prob, mut_prob=mutation_rate, elitism=elitism)

# Get the best individual
best_individual = population.individuals[population.fitness.index(min(population.fitness))]
best_image = best_individual.draw()
best_image.show()
